Рассмотрим уравнение 

\begin{equation*}
i\frac{\partial a}{\partial {t}} = i\frac{\partial a}{\partial {x}} - \dfrac{g}{4} |a|^2 a - i \dfrac{g^2}{32 M_0^2}  {|a|^2 \dfrac{\partial |a|^2} {\partial x} a} - \eta \dfrac{\partial^2 a} {\partial x^2} + M_0^2 \eta a 
 \:.
\end{equation*} 

Построим численное решение с применением метода split-step.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from scipy.fft import fft, ifft

Параметры ($M, g, \eta$ можно менять), но пока зафиксируем: 

In [3]:
M = 1 
g = 1
eta = 0.01*0
kappa=1
q = 1

Функции, которые используются для численного интегрирования на линейном и нелинейном шагах. https://numpy.org/doc/stable/reference/routines.fft.html -- про Фурье в Питоне ($a_k \propto e^{-ikx}$)

In [4]:
def LinearStep(dt, psi, kx, eta, kappa, M):
    psiNew = ifft(np.exp( ( - 1j * dt )* (  - kappa* kx + eta* kx**2 + M**2*eta) )* fft( psi ) )
    return psiNew

def NonlinearStep(dt, psi, kx, M, g, q):
    rho = abs(psi) ** 2
    drho_dx = q*ifft( 1j * kx * fft(rho))
    psiNew = np.exp( ( 1j * dt )* ( g / 4 )* rho )* psi
    psiNew = np.exp( ( - dt )* ( g**2 / ( 32 * M**2 ) )* rho* drho_dx )* psiNew
    return psiNew

Можно менять шаг

In [5]:
(0.05**2)/2

0.0012500000000000002

In [6]:
Nt = 10000
t = np.zeros(Nt)
dx = 0.1/2

dt = (dx**2)/2
Nx = 2**12
Lx = dx*Nx
x = np.zeros(Nx)
kx = np.zeros(Nx)
gridx=np.linspace(0, Nx-1, Nx)
gridt=np.linspace(0, Nt-1, Nt)
t=gridt*dt
for ii in gridx:
    x[round(ii)] = ( round(ii) + 1 - (1 + (Nx / 2))) * dx
for ii in gridx:
    kx[round(ii)] = ( round(ii) + 1 - ( 1 + Nx / 2 ) ) * ( 2 * math.pi / Lx );
kx = scipy.fft.fftshift (kx);

Начальное распределение поля

In [7]:
amplitude = 0.6
width = 5
xStart = 0
exponent=-(((x - xStart) / width) ** 2)
psi = amplitude * np.exp(exponent)

Интегрируем уравнение

In [8]:
Psi_numerical_re = np.zeros((len(t)+1, len(x)))
Psi_numerical_im = np.zeros((len(t)+1, len(x)))
Psi_numerical = np.zeros((len(t)+1, len(x)))

Psi_numerical_re[0, :] = psi.real 
Psi_numerical_im[0, :] = psi.imag 
for it in gridt: 
    itc=round(it)+1
    psi = LinearStep(0.5 * dt, psi, kx, eta, kappa, M)
    psi = NonlinearStep(1.0 * dt, psi, kx, M, g, q)
    psi = LinearStep(0.5 * dt, psi, kx, eta, kappa, M)
    Psi_numerical_re[itc, :] = psi.real 
    Psi_numerical_im[itc, :] = psi.imag 
Psi_numerical =  Psi_numerical_re + 1j*Psi_numerical_im
a = np.transpose(Psi_numerical)

С применением численного дифференцирования проверим, что выполняется

\begin{equation*}
i\frac{\partial a}{\partial {t}} = i\frac{\partial a}{\partial {x}} - \dfrac{g}{4} |a|^2 a - i \dfrac{g^2}{32 M_0^2}  {|a|^2 \dfrac{\partial |a|^2} {\partial x} a} - \eta \dfrac{\partial^2 a} {\partial x^2} + M_0^2 \eta a 
 \:.
\end{equation*}.

Трешхолд меньше nd не имеет смысл рассматривать?

In [9]:
# dt_a = np.gradient(a, axis=1)/dt;
# dx_a = np.gradient(a, axis=0)/dx;
# dx_abs_a_2 = np.gradient(abs(a)**2, axis=0)/dx; 
# dxx_a =  np.gradient(dx_a, axis=0)/dx;
# # dt_a = ps.FiniteDifference(axis=1)._differentiate(a, t=dt)
# # dx_a = ps.FiniteDifference(axis=0)._differentiate(a, t=dx)
# # dx_abs_a_2 = ps.FiniteDifference(axis=0)._differentiate(abs(a)**2, t=dx)
# # dxx_a = ps.FiniteDifference(axis=0)._differentiate(dx_a, t=dx)
# nd = abs(-1j*dt_a+1j*kappa*dx_a-g/4*abs(a)**2*a-1j*g**2/(32*(M**2))*(abs(a)**2)*a*dx_abs_a_2-eta*dxx_a+M**2*eta*a)
# print(np.amax(nd))
# # plt.subplot(1, 3, 1)
# # plt.pcolormesh(np.log(abs(num_difference)))
# # plt.xlabel('t', fontsize=16)
# # plt.ylabel('x', fontsize=16)
# # plt.title(r'$|a(x, t)|^2$', fontsize=16)

Выделяем действительную и мнимую часть, модуль и фазу.

In [10]:
a_r = a.real
a_i = a.imag
rho = abs(a)
phi_0 = np.angle(a)
phi = 0*np.angle(a)

Зануляем фазу там, где почти нет поля

In [11]:
for ii in gridx:
    for it in gridt: 
        if rho [round(ii), round(it)] > 0.0001:
            phi [round(ii), round(it)] =  phi_0 [round(ii), round(it)] 

Стоим всякие штуки

In [12]:
# plt.subplot(1, 2, 1)
# plt.pcolormesh(abs(a)**2)
# plt.xlabel('t', fontsize=16)
# plt.ylabel('x', fontsize=16)
# plt.title(r'$|a(x, t)|^2$', fontsize=16)
# plt.subplot(1, 2, 2)
# plt.plot(a_r[:, Nt-1], label = 're')
# plt.plot(a_i[:, Nt-1], label = 'im')
# plt.plot(rho[:, Nt-1], label = 'abs')
# plt.plot(phi_0[:, Nt-1], label = 'phi0')
# plt.plot(phi[:, Nt-1], label = 'phi')
# plt.legend()
# plt.title(r'$Field~at~t=end$', fontsize=16)

Text(0.5, 1.0, '$Field~at~t=end$')

: 

: 

Далее применяем методы pysindy

Grid Search?

In [ ]:
# !pip install pysindy
import pysindy as ps

In [ ]:
# эта ячейка пригодится, когда будем подгружать реальные данные 
# x_matlab = np.loadtxt('x.dat')
# t_matlab = np.loadtxt('t.dat')
# dt_matlab=abs(t[1]-t[2]);
# dx_matlab=abs(x[1]-x[2]);
# re_a = np.loadtxt('Solutionre_mNLSeq.dat')
# im_a = np.loadtxt('Solutionim_mNLSeq.dat')
# a_transpose=re_a+1j*im_a
# a = np.transpose(a_transpose)

Мы можем восстанавливать это уравнение 
$$
\begin{aligned}
& {\color{red}{\quad \frac{\partial a_R}{\partial t}=\frac{\partial a_R}{\partial x}-\frac{g}{4}\left[a_I^3+a_R^2 a_I\right]}}-\frac{g^2}{16 M_0^2}\left[a_R^4 \frac{\partial a_R}{\partial x}+a_{ I}^2 a_R^2 \frac{\partial a_R}{\partial x}+\right. \\
& \left.+a_R^3 a_I \frac{\partial a_I}{\partial x}+a_I^3 a_R \frac{\partial a_I}{\partial x}\right]-\eta \frac{\partial^2 a_I}{\partial x^2}+M_0^2 \eta a_I
\end{aligned}
$$
$$
\begin{aligned}
&{\color{red}{ \frac{\partial a_I}{\partial t}=\frac{\partial a_I}{\partial x}+\frac{g}{4} a_R^3+\frac{g}{4} a_I^2 a_R}}-\frac{g^2}{16 M_0^2}\left[a_I^3 a_R \frac{\partial a_R}{\partial x}+a_R^3 a_I \frac{\partial a_R}{\partial x}+\right. \\
& \left.+a_I^4 \frac{\partial a_I}{\partial x}+a_R^2 a_I^2 \frac{\partial a_I}{\partial x}\right]+\eta \frac{\partial^2 a_R}{\partial x^2}-M_0^2 \eta a_R
\end{aligned}
$$

или это 

\begin{aligned}
& a=\rho e^{i \varphi} \\
& \text { Re: } \frac{\partial \varphi}{\partial t}= \frac{\partial \varphi}{\partial x}+\frac{g}{4} \rho^2+\eta \frac{\partial^2 \rho}{\partial x^2}/\rho-\eta\left(\frac{\partial \varphi}{\partial x}\right)^2 -M_0^2 \eta  \\
& \text { Im: } \frac{\partial \rho}{\partial t}=\frac{\partial \rho}{\partial x}-\frac{g^2}{16 M_0^2} \rho^4 \frac{\partial \rho}{\partial x}- 2 \eta  \frac{\partial \varphi }{\partial x} \frac{\partial \rho}{\partial x}-\eta \frac{\partial^2 \varphi}{\partial x^2} \rho \\
&
\end{aligned}

Но уравнение для модуля и фазы будет работать только если $\eta=0$, потому что иначе там деление на функцию

In [ ]:
#для первого уравнения
# u = np.zeros((len(x), len(t)+1, 2))
# u[:, :,  0] = a_r
# u[:, :,  1] = a_i
#для второго уравнения
u = np.zeros((len(x), len(t)+1, 2))
u[:, :,  0] = rho
u[:, :,  1] = phi

In [ ]:
#https://github.com/dynamicslab/pysindy/blob/master/examples/12_weakform_SINDy_examples.ipynb
# тут нужно обновить библиотеку функций
library_functions = [
    lambda x: x,

    lambda x: x * x,
    lambda x, y: x * y,

    lambda x: x * x * x,
    lambda x, y: x * x * y,
    lambda x, y: x * y * y,
    lambda y: y * y * y,

    lambda x: x * x * x * x,
    lambda x, y: x * x * x * y,
    lambda x, y: x * x * y * y,
    lambda x, y: x * y * y * y,
    lambda y: y * y * y * y,

]
library_function_names = [
    lambda x: 'x',

    lambda x: 'x * x',
    lambda x, y: 'x * y',

    lambda x: 'x * x * x',
    lambda x, y: 'x * x * y',
    lambda x, y: 'x * y * y',
    lambda y: 'y * y * y',

    lambda x: 'x * x * x * x',
    lambda x, y: 'x * x * x * y',
    lambda x, y: 'x * x * y * y',
    lambda x, y: 'x * y * y * y',
    lambda y: 'y * y * y * y',
]

In [ ]:
np.random.seed(100)
X, T = np.meshgrid(x, t, indexing='ij')
XT = np.transpose([X, T], [1, 2, 0])
pde_lib = ps.PDELibrary(library_functions=library_functions, 
                        function_names=library_function_names, 
                        derivative_order=2, spatial_grid=x,
                        include_bias=False,
                        is_uniform=True)


# Fit and predict with the non-weak model
# opt = ps.SR3(threshold=0.1, thresholder='l0', 
#              tol=1e-10, normalize_columns=True, 
#              max_iter=1000)

In [ ]:
# Fit the equation, weak form style!
optimizer = ps.STLSQ(threshold=0.001, alpha=1e-5, normalize_columns=True)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(u, quiet=True)
model.print()

\begin{aligned}
& a=\rho e^{i \varphi} \\
& \text { Re: } \frac{\partial \varphi}{\partial t}= \frac{\partial \varphi}{\partial x}+\frac{g}{4} \rho^2+\eta \frac{\partial^2 \rho}{\partial x^2}/\rho-\eta\left(\frac{\partial \varphi}{\partial x}\right)^2 -M_0^2 \eta  \\
& \text { Im: } \frac{\partial \rho}{\partial t}=\frac{\partial \rho}{\partial x}-\frac{g^2}{16 M_0^2} \rho^4 \frac{\partial \rho}{\partial x}- 2 \eta  \frac{\partial \varphi }{\partial \rho} \frac{\partial x}{\partial x}-\eta \frac{\partial^2 \varphi}{\partial x^2} \rho \\
&
\end{aligned}